## 전체파일받아서 처리 
- 파일 전체를 받아서, 그거에 상응하는 파일을 처리함. 

In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
from operator import itemgetter

In [3]:
import re

In [4]:
jsonDir = './data/match_timeline'
matchDir = './data/match'

In [5]:
fileOutputDir = './fileOutputs/'

In [6]:
listDir = os.listdir(jsonDir)

In [7]:
len(listDir)

12697

In [8]:
listDir

['KR_7349561394.json',
 'KR_7338313389.json',
 'KR_7347748748.json',
 'KR_7349749926.json',
 'KR_7335044071.json',
 'KR_7349686650.json',
 'KR_7335191209.json',
 'KR_7343265953.json',
 'KR_7342471918.json',
 'KR_7347264281.json',
 'KR_7348820279.json',
 'KR_7347837892.json',
 'KR_7348894670.json',
 'KR_7349878855.json',
 'KR_7344353899.json',
 'KR_7346775137.json',
 'KR_7347181328.json',
 'KR_7327096029.json',
 'KR_7347307567.json',
 'KR_7344847033.json',
 'KR_7347097937.json',
 'KR_7348261857.json',
 'KR_7347758748.json',
 'KR_7348127761.json',
 'KR_7345770195.json',
 'KR_7349009450.json',
 'KR_7336368241.json',
 'KR_7344439807.json',
 'KR_7348477920.json',
 'KR_7340931547.json',
 'KR_7338501317.json',
 'KR_7346668488.json',
 'KR_7344402741.json',
 'KR_7328219085.json',
 'KR_7345738704.json',
 'KR_7347303461.json',
 'KR_7344022960.json',
 'KR_7339902304.json',
 'KR_7347190885.json',
 'KR_7348997781.json',
 'KR_7346070918.json',
 'KR_7343378916.json',
 'KR_7333836986.json',
 'KR_734590

In [9]:
resultFrame = list()

In [10]:
def getChampionStats(infoFrameData, championDf, matchId):
    for i in range(0, len(infoFrameData),1):
        for pN in range(1, 9, 1):
            resultFrame.append({**infoFrameData[i]['participantFrames'][str(pN)]['championStats'],\
                       'participantId' : infoFrameData[i]['participantFrames'][str(pN)]['participantId'],\
                       'currentGold': infoFrameData[i]['participantFrames'][str(pN)]['currentGold'],\
                        **infoFrameData[i]['participantFrames'][str(pN)]['damageStats'],\
                        'goldPerSecond':infoFrameData[i]['participantFrames'][str(pN)]['goldPerSecond'],\
                        'jungleMinionKilled': infoFrameData[i]['participantFrames'][str(pN)]['jungleMinionsKilled'],\
                        'level': infoFrameData[i]['participantFrames'][str(pN)]['level'],\
                        'minionKilled': infoFrameData[i]['participantFrames'][str(pN)]['minionsKilled'],\
                        'timeEnemySpendControlled':infoFrameData[i]['participantFrames'][str(pN)]['timeEnemySpentControlled'],\
                        'totalGold':infoFrameData[i]['participantFrames'][str(pN)]['totalGold'],\
                        'xp':infoFrameData[i]['participantFrames'][str(pN)]['xp']})
    
    resultFrameDf = pd.DataFrame(resultFrame)
    print(championDf)
    participantStat = pd.merge(championDf[['participantId','championId']], resultFrameDf, on = 'participantId')
    participantStat['matchId'] = matchId   
    participantStat.to_csv(os.path.join(fileOutputDir,f'{matchId}_CHAMPION_STAT_PER_TIMESTAMP.csv')) 
    

In [11]:
def convertToDf(bucketData,  matchId):
    
    bucketDf = pd.DataFrame(bucketData)
    bucketDf['matchId'] = matchId
    
    return bucketDf

In [31]:
def getgameLogData(matchLineData, championDf, matchId):
    
    itemSoldBucket = list()
    gameEndBucket = list()
    levelUpBucket = list()
    skilLevelUpBucket = list()
    wardKillBucket = list()

    for i in range(0, len(matchLineData),1):
        elementData = matchLineData[i]['events']
        for e in elementData : 

                if e['type'] == "ITEM_SOLD":
                        itemSoldBucket.append(elementData)
    
                if e['type'] == "GAME_END":
                        gameEndBucket.append(elementData)
    
                if e['type'] == "LEVEL_UP":
                        levelUpBucket.append(elementData)
    

                if e['type'] == "SKILL_LEVEL_UP":
                        skilLevelUpBucket.append(elementData)
    
                if e['type'] ==  "WARD_KILL":
                        wardKillBucket.append(elementData)
    

    itemSoldDf = convertToDf(itemSoldBucket, matchId)
    gameEndDf = convertToDf(gameEndBucket, matchId)
    levelUpDf = convertToDf(levelUpBucket, matchId)
    skillLevelUpDf = convertToDf(skilLevelUpBucket, matchId)
    wardKillDf = convertToDf(wardKillBucket, matchId)

    #각각 csv로 변환

    itemSoldDf.to_csv(os.path.join(fileOutputDir, '{matchId}_ITEM_SOLD.csv'))
    gameEndDf.to_csv( os.path.join(fileOutputDir,f'{matchId}_GAME_END.csv'))
    levelUpDf.to_csv( os.path.join(fileOutputDir,f'{matchId}_LEVEL_UP.csv'))
    skillLevelUpDf.to_csv( os.path.join(fileOutputDir,f'{matchId}_SKILL_LEVEL_UP.csv'))
    wardKillDf.to_csv( os.path.join(fileOutputDir,f'{matchId}_WARD_KILL.csv'))

    #return  itemSoldDf, gameEndDf, levelUpDf, skillLevelUpDf, wardKillDf

In [29]:
def getChampionData(matchData):
    championList = list()
    print(matchData)
    matchId = matchData['metadata']['matchId'] 
    matchId = matchId.split('_')[1]
    for championData in matchData['info']['participants']:
        values = itemgetter(*['participantId', 'championId','championName'])(championData)
        championList.append((matchId,)+ values  )
    
    championDataDf= pd.DataFrame(championList, columns = ['matchId', 'participantId', 'championId','championName'], )
    championDataDf.to_csv(os.path.join(fileOutputDir, f'{matchId}_CHAMPION_PARTICIPANT_ID.csv'))

    return matchId, championDataDf

In [14]:
listDir

['KR_7349561394.json',
 'KR_7338313389.json',
 'KR_7347748748.json',
 'KR_7349749926.json',
 'KR_7335044071.json',
 'KR_7349686650.json',
 'KR_7335191209.json',
 'KR_7343265953.json',
 'KR_7342471918.json',
 'KR_7347264281.json',
 'KR_7348820279.json',
 'KR_7347837892.json',
 'KR_7348894670.json',
 'KR_7349878855.json',
 'KR_7344353899.json',
 'KR_7346775137.json',
 'KR_7347181328.json',
 'KR_7327096029.json',
 'KR_7347307567.json',
 'KR_7344847033.json',
 'KR_7347097937.json',
 'KR_7348261857.json',
 'KR_7347758748.json',
 'KR_7348127761.json',
 'KR_7345770195.json',
 'KR_7349009450.json',
 'KR_7336368241.json',
 'KR_7344439807.json',
 'KR_7348477920.json',
 'KR_7340931547.json',
 'KR_7338501317.json',
 'KR_7346668488.json',
 'KR_7344402741.json',
 'KR_7328219085.json',
 'KR_7345738704.json',
 'KR_7347303461.json',
 'KR_7344022960.json',
 'KR_7339902304.json',
 'KR_7347190885.json',
 'KR_7348997781.json',
 'KR_7346070918.json',
 'KR_7343378916.json',
 'KR_7333836986.json',
 'KR_734590

In [32]:
for e in tqdm(listDir):
    
    matchPath = os.path.join(matchDir, e)
    matchTimelineDir = os.path.join(jsonDir, e)
    print(matchPath)
    print(matchTimelineDir)
    if os.path.isfile(matchPath) and os.path.isfile(matchTimelineDir):
        print(f'{e} is processing')
        with open(matchTimelineDir) as f:
            data = json.load(f)
        with open(matchPath ) as f:
            matchData = json.load(f)  
        infoFrameData = data['info']['frames'] 
        matchId, championDf = getChampionData(matchData)
        getChampionStats(infoFrameData, championDf, matchId)
        getgameLogData(infoFrameData, championDf, matchId)
    else: 
        continue
    
            

  0%|          | 0/12697 [00:00<?, ?it/s]

./data/match/KR_7349561394.json
./data/match_timeline/KR_7349561394.json
KR_7349561394.json is processing
{'metadata': {'dataVersion': '2', 'matchId': 'KR_7349561394', 'participants': ['IplSjVgSQyfH2Xo2cEip03LM4NDiz73p_2SLmtBhS-0bbsoaNXJbfINKaTdzNXVelVHG4jYjcoCt7A', 'KtPVMynJUA0lPVLagX6tLT88VFkPsdb9FappUVJ5NEyNjiknu5mX5YRd42YtwhIn9dROien6nI6JMg', '7zBEDj3buDDMmDfal7NsrtzYVFjjz3WT8boEmKccKj26okpxxePol4YdeX5L-THgwH5jAt5G3-Fhcg', 'oBAaSP6t3N0BPk0M9FQnf-lzquvj6dRKZuFWWzbndHhLw8pMgMRvakmjn1WBlsYNbTK2cVZILaJKzg', 'GgzD5d4yge634pn4Tn2V4mLea4vGCoJouO-Zh0V18eeVw45i5KNaT_AhAwXGU4O2Ooqzia6D8dDkZQ', 'SdSvdckAcgL8OauAmue5GTWslcdtK-enOz0NVKsT3AgcNB38ruBCpfWyeH3exEHiXinOAALyHi1NxQ', 'JrtYQUr6gdWdX5z1XjJEhikOSDKnMKkoeNnmSJEj1OO7feZYww5DIor3gC1rAoKbZFDmumyAsJEngw', 'qaSly9NaJhLA7kPSi_EFgfTl7aykReOT0fjlg4zrhnPJ-U-CfxETvjOXRFUg_58i9oKkEttTdeRIzw', '_MqlZTkpz39pLCpM_ldyguTAlolrCSr21PcjOgzq1VKRN5YcihGiESa4yTqrvSYNxEUnX8yAo4a_wg', 'DyVG-3BNiO8U1eMC6-CNhMYYFiWUxaYdIbFp5BuCDcXv-fgdEN30mDwQ2RGxoyYKkSNcDLKM2p16

UnboundLocalError: cannot access local variable 'skillLevelUpDf' where it is not associated with a value

In [19]:
#함수 짜는 친구들 짜기 
#주머니를 일단 하나씩 만들어서 넣으면 되지 않을까? 
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.6 MB/s eta 0:00:00a 0:00:01
